In [1]:
import os
import torch

from tqdm.notebook import tqdm
%matplotlib notebook 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80


from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

#from src.mesh.cube import NetCube, to_vertices
from src.mesh.supershape import Supershape
from argparse import Namespace


device = torch.device('cuda')

In [2]:
# Load the dolphin mesh.
trg_obj = os.path.join('./data/scenes/centered/meshes/centered.obj')

# We read the target 3D model using load_obj
verts, faces, aux = load_obj(trg_obj)

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We scale normalize and center the target mesh to fit in a sphere of radius 1 centered at (0,0,0). 
# (scale, center) will be used to bring the predicted mesh to its original center and scale
# Note that normalizing the target mesh, speeds up the optimization but is not necessary!
# center = verts.mean(0)
# verts = verts - center
# scale = max(verts.abs().max(0)[0])
# verts = verts / scale

# We construct a Meshes structure for the target mesh
trg_mesh = Meshes(verts=[verts], faces=[faces_idx])#.to(device)
trg_mesh

In [41]:
n = 64
supershape = Supershape(n).to(device)
#cube = NetCube(n, opt, kernel=7, sigma=2).cuda()
#cube.load_state_dict(torch.load('./data/net_fitted_32.pth'))


#radii = torch.rand(3, device=v_ref.device, requires_grad=True)
v, f = supershape()
# n_ref = compute_face_normals(v_ref, f_ref)
print([f.shape for f in [v, f]])

src_mesh =  Meshes(verts=[v], faces=[f])
src_mesh

[torch.Size([24576, 3]), torch.Size([49148, 3])]


In [42]:
sample_trg = sample_points_from_meshes(trg_mesh, 5000)
new_src_mesh = src_mesh.offset_verts(v)
sample_src = sample_points_from_meshes(new_src_mesh, 5000)
sample_trg, sample_src

(tensor([[[ 0.8055,  0.0555, -0.0248],
          [-0.2172, -0.1093, -0.5367],
          [ 0.2522, -0.3929, -0.5367],
          ...,
          [ 0.2304, -0.7286,  0.2797],
          [ 0.3879, -0.0278,  0.4920],
          [-0.5514,  0.9247, -0.1477]]], device='cuda:0'),
 tensor([[[ 1.7930,  0.2350,  0.8537],
          [ 1.1914, -1.3570, -0.8593],
          [ 1.0953, -0.1285,  1.6683],
          ...,
          [ 1.2362,  0.6526, -1.4302],
          [-1.5262,  1.0125,  0.8030],
          [-0.5473,  1.6848, -0.9282]]], device='cuda:0',
        grad_fn=<IndexPutBackward0>))

In [43]:
#optimizer = torch.optim.SGD(cube.parameters(), lr=1.0, momentum=0.9)
optimizer = torch.optim.Adam(supershape.parameters(), lr=1.0)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1.0
    weight_decay: 0
)

In [44]:
# Number of optimization steps
Niter = 20000
# Weight for the chamfer loss
w_chamfer = 1.0 
# Weight for mesh edge loss
w_edge = 1.0 
# Weight for mesh normal consistency
w_normal = 0.01 
# Weight for mesh laplacian smoothing
w_laplacian = 0.1 
# Plot period for the losses
plot_period = 250
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []



for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()
    
    v, f = supershape()
    # n_ref = compute_face_normals(v_ref, f_ref)
    #print([f.shape for f in [v, f]])

    new_src_mesh = Meshes(verts=[v], faces=[f])
    # Deform the mesh
    # new_src_mesh = src_mesh.offset_verts(v)
    
    # We sample 5k points from the surface of each mesh 
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)
    
    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    
    #if False:
    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian
    #loss = loss_chamfer
    # Print the losses
    loop.set_description('total_loss = %.6f' % loss)
    
    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
#     edge_losses.append(float(loss_edge.detach().cpu()))
#     normal_losses.append(float(loss_normal.detach().cpu()))
#     laplacian_losses.append(float(loss_laplacian.detach().cpu()))
    
    # Plot mesh
    #if i % plot_period == 0:
    #    plot_pointcloud(new_src_mesh, title="iter: %d" % i)
        
    # Optimization step
    loss.backward()
    optimizer.step()
    
chamfer_losses    

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [45]:
supershape.cpu().plot()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.049210…

In [ ]:
vert =  cube.get_start().to(device) + v
meshplot.plot(vert.detach().cpu().numpy(), f.cpu().numpy())

In [ ]:
vert